# Hecho servicios por hora

In [5]:
import datetime
import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine, text

# Añadir conexión a la base de datos

In [6]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Crear la tabla de hechos en la base de datos ETL

In [7]:
create_table_query = """
CREATE TABLE IF NOT EXISTS hecho_servicios_fases (
    servicio_id INTEGER NOT NULL,
    cliente_id INTEGER NOT NULL,
    sede_id INTEGER NOT NULL,
    mensajero_id INTEGER NOT NULL,
    novedad_id INTEGER,
    fecha_solicitud_id INTEGER NOT NULL,
    hora_solicitud_id INTEGER NOT NULL,
    fecha_cierre_id INTEGER,
    hora_cierre_id INTEGER,
    tiempo_entrega_minutos INTEGER,
    PRIMARY KEY (servicio_id)
);
"""

# Ejecutar la creación de la tabla en la base de datos ETL

In [8]:
with etl_conn.connect() as conn:
    conn.execute(text(create_table_query))
    print("Tabla de hechos `hechos_servicios_por_fase` creada con éxito.")

Tabla de hechos `hechos_servicios_por_fase` creada con éxito.


# Extracción de datos

In [9]:
query_servicios = """
SELECT
    s.id AS servicio_id,
    s.cliente_id,
    s.mensajero_id,
    s.fecha_solicitud,
    EXTRACT(HOUR FROM s.hora_solicitud) * 100 + EXTRACT(MINUTE FROM s.hora_solicitud) AS hora_solicitud_id,
    es.fecha AS fecha_cierre,
    EXTRACT(HOUR FROM es.hora) * 100 + EXTRACT(MINUTE FROM es.hora) AS hora_cierre_id
FROM
    public.mensajeria_servicio s
JOIN
    public.mensajeria_estadosservicio es ON s.id = es.servicio_id
WHERE
    es.estado_id = (SELECT id FROM public.mensajeria_estado WHERE nombre = 'Cerrado')
"""


# Leer los datos desde la base de datos de origen

In [19]:
df_servicios = pd.read_sql(query_servicios, ryf_conn)

df_servicios.head()

,servicio_id,cliente_id,mensajero_id,fecha_solicitud,hora_solicitud_id,fecha_cierre,hora_cierre_id


# Calcular el tiempo de entrega en minutos y preparar datos para insertar

In [11]:
df_servicios['tiempo_entrega_minutos'] = (
        (pd.to_datetime(df_servicios['fecha_cierre']) + pd.to_timedelta(df_servicios['hora_cierre_id'], unit='m') -
         (pd.to_datetime(df_servicios['fecha_solicitud']) + pd.to_timedelta(df_servicios['hora_solicitud_id'], unit='m'))
         ).dt.total_seconds() / 60
).astype(int)

# Preparar los datos para la carga en la tabla de hechos

In [12]:
df_hechos = df_servicios[['servicio_id', 'cliente_id', 'mensajero_id', 'fecha_solicitud',
                          'hora_solicitud_id', 'fecha_cierre', 'hora_cierre_id', 'tiempo_entrega_minutos']]


# Insertar los datos en la tabla de hechos en la base de datos ETL

In [13]:
df_hechos.to_sql('hechos_servicios_por_fase', etl_conn, if_exists='append', index=False)
print("Datos insertados en la tabla `hechos_servicios_por_fase`.")


Datos insertados en la tabla `hechos_servicios_por_fase`.


# Consulta para calcular el tiempo promedio de entrega

In [16]:
query_promedio_entrega = """
SELECT AVG(tiempo_entrega_minutos) AS tiempo_promedio_entrega
FROM hechos_servicios_por_fase;
"""

# Ejecutar la consulta y mostrar el resultado

In [18]:
# Ejecutar la consulta y mostrar el resultado
with etl_conn.connect() as conn:
    resultado = conn.execute(text(query_promedio_entrega)).fetchone()
    tiempo_promedio_entrega = resultado[0]  # Acceder al primer elemento del tuple
    print(f"Tiempo promedio de entrega: {tiempo_promedio_entrega} minutos")


Tiempo promedio de entrega: None minutos
